
# Google Landmark Recognition 2021 

## Importing Library¶


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image 
import matplotlib.image as mpimg
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.layers import Input, Conv2D,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import clear_output
import random, re, math
import pandas as pd 
import keras.backend as K
from sklearn.model_selection import train_test_split


## Making a dataframe of label_train


In [ ]:
df=pd.read_csv('../input/landmark-recognition-2021/train.csv')
df.head()

## Adding path_columns from train images pathes

### ***By adding path to our dataframe we can use this dataframe in Tensorflow directly!***

In [ ]:
%%time
df['path']=['../input/landmark-recognition-2021/train/'+id[0]+'/'+id[1]+'/'+id[2]+'/'+id+'.jpg' for id in df['id']]
df.head()

## Label types and null values check¶

In [ ]:
df.info()

## label number check

In [ ]:
df['landmark_id'].nunique()

In [ ]:
df['landmark_id'].value_counts()

In [ ]:
df['landmark_id'].describe()

## Encoding the landmark_id

In [ ]:

df['id_counts'] = df.landmark_id.value_counts().loc[df.landmark_id.values].values
id_map = df.sort_values(by='id_counts').landmark_id.drop_duplicates().reset_index(drop=True)
id_dict = {id_map.loc[x]:81312-x for x in range(81313)}
df['encode_id'] = df.landmark_id.apply(lambda x: id_dict[x])
df.head()

In [ ]:
df['landmark_id'].describe()

In [ ]:
df['encode_id'].describe()

## Let's take a look at some image examples and their correponding Landmark_id

In [ ]:
plt.subplots(3, 4, figsize=(160, 120))
for i in range(12):
    sp = plt.subplot(3, 4, i + 1)
    sp.axis('Off')
    im = mpimg.imread(df.iloc[i][2])
    plt.imshow(im)
    plt.title(f'landmark_id:{df.iloc[i][1]} ', 
                                     fontweight ="bold",fontsize=100)
    

## Configuration

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
IMAGE_SIZE = [256, 256]
EPOCHS = 40
SEED = 24
BATCH_SIZE = 32 

In [ ]:
def flip(image,label):
    
    image = tf.image.flip_left_right(image)
    
    return image,label



def rotate(image,label):

    rot = 15. * tf.random.normal([1],dtype='float32')
    rotation = math.pi * rot / 180.
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    m = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    img=tf.reshape(d,[DIM,DIM,3])
    return img,label
#tf.cast(np.array(img), dtype=tf.float32)



def read_image_and_label(image_path, label=None,resize=IMAGE_SIZE):
    
    image=tf.io.read_file(image_path)
    image=tf.image.decode_jpeg(image, channels=3)
    image=tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, dtype=tf.float32)/255.
    if not label is None:
        return image, label
    return image




def get_training_dataset(df):
    
    training_dataset = tf.data.Dataset.from_tensor_slices((df["path"].values, df["encode_id"].values))
    training_dataset = training_dataset.map(read_image_and_label,num_parallel_calls=AUTO)
    training_dataset = training_dataset.map(flip,num_parallel_calls=AUTO)
    training_dataset = training_dataset.map(rotate,num_parallel_calls=AUTO)
    training_dataset = training_dataset.shuffle(1000, reshuffle_each_iteration=True)
    training_dataset = training_dataset.batch(BATCH_SIZE)
    training_dataset = training_dataset.prefetch(AUTO)

    return training_dataset


def get_validation_dataset(df,batch_size=16):
  
  validation_dataset = tf.data.Dataset.from_tensor_slices((df["path"].values, df["encode_id"].values))
  validation_dataset = validation_dataset.map(read_image_and_label)
  validation_dataset = validation_dataset.batch(BATCH_SIZE)
  

  return validation_dataset


def get_test_dataset(images,batch_size=16):
  
  test_dataset = tf.data.Dataset.from_tensor_slices((images))
  test_dataset = test_dataset.map(read_image_and_label)
  test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

  return test_dataset

In [ ]:
tr_dataset = get_training_dataset(train_test_split(df, random_state=SEED, test_size=.25)[0])
val_dataset = get_validation_dataset(train_test_split(df, random_state=SEED, test_size=.25)[1])

## Display Example Augmentation¶¶


In [ ]:
row = 2; col = 4;
all_elements = tr_dataset.unbatch()
one_element = tf.data.Dataset.from_tensors( next(iter(all_elements)) )
augmented_element = one_element.repeat().map(rotate).batch(row*col)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
        
    plt.show()
    break

## Build a model

### Since we have a large dataset,a complex model works better than a simple one but as we will have a long runtime I used a simple model with just 2 epochs in this notebook!

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


def conv_block(filters, inputs):
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.MaxPool2D()(x)

    return outputs


def dense_block(units, dropout_rate, inputs):
    x = layers.Dense(units, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dropout(dropout_rate)(x)

    return outputs

In [ ]:
def build_model():
    inputs = keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(inputs)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = layers.Dense(81313, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:

model = build_model()
model.summary()

## Model training

In [ ]:

# Compile Model. 
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

# Train the Model
#history = model.fit_generator( tr_dataset, epochs=2,validation_data=val_dataset)

